In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os,sys,inspect, string
import re
import csv
import importlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [12]:
from config import PROJECT_ROOT_DIR
from src.dataset_getters import *
from src.signal_utils import get_raw_signal_from_file

In [68]:
LABELS_PATH = get_kaggle_labels_path('a')
LABELS_PATH

'/home/justyna/WORKSPACE/heartbeat_classification/data/raw/kaggle/set_a.csv'

In [78]:
labels = pd.read_csv(LABELS_PATH)
labels

,Unnamed: 0,Unnamed: 0.1,set,filename,label,new_label
0,0,0,a,artifact__201012172012.wav,artifact,artifact
1,1,1,a,artifact__201105040918.wav,artifact,artifact
2,2,2,a,artifact__201105041959.wav,artifact,artifact
3,3,3,a,artifact__201105051017.wav,artifact,artifact
4,4,4,a,artifact__201105060108.wav,artifact,artifact


In [83]:
labels.drop(labels="Unnamed: 0.1",axis=1, inplace=True)

In [85]:
labels.drop(labels="Unnamed: 0",axis=1, inplace=True)

In [87]:
labels.head()

,set,filename,label,new_label
0,a,artifact__201012172012.wav,artifact,artifact
1,a,artifact__201105040918.wav,artifact,artifact
2,a,artifact__201105041959.wav,artifact,artifact
3,a,artifact__201105051017.wav,artifact,artifact
4,a,artifact__201105060108.wav,artifact,artifact


In [32]:
labels.sort_values(by='filename', inplace=True)
labels = labels.reset_index(drop=True)
labels['new_label'] = pd.Series([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 0, 2, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [34]:
labels.loc[labels['new_label'] == 0, 'new_label'] = 'artifact'
labels.loc[labels['new_label'] == 1, 'new_label'] = 'normal'
labels.loc[labels['new_label'] == 2, 'new_label'] = 'murmur'

In [36]:
labels.rename(index=str, columns={"label": "original_label"})

,set,filename,original_label,new_label
0,a,artifact__201012172012.wav,artifact,artifact
1,a,artifact__201105040918.wav,artifact,artifact
2,a,artifact__201105041959.wav,artifact,artifact
3,a,artifact__201105051017.wav,artifact,artifact
4,a,artifact__201105060108.wav,artifact,artifact
5,a,artifact__201105061143.wav,artifact,artifact
6,a,artifact__201105190800.wav,artifact,artifact
7,a,artifact__201105280851.wav,artifact,artifact
8,a,artifact__201106010559.wav,artifact,artifact
9,a,artifact__201106010602.wav,artifact,artifact


In [88]:
labels.to_csv(LABELS_PATH, index=False)